<img src="Archivos/miad4.png" width=800x>

# Modificar bases de datos  

Al finalizar la exploración de una base de datos procedemos a modificarla. En este tutorial expandimos la explicación de los métodos de la librería `pandas` vistos anteriormente y presentamos métodos adicionales para integrarlos en procesos de modificación de bases de datos. 

## Requisitos

Para desarrollar este tutorial necesitarás:

* Importar archivos de texto.
* Implementar filtrado condicional y filtrado numérico.
* Utilizar los métodos básicos de los `Series` y los `DataFrame`.

## Objetivos

Al final de este tutorial podrás:

**1.** Procesar archivos en formatos especiales. <br>
**2.** Modificar la estructura y el contenido de un `DataFrame`.

## 1. Procesamiento de archivos en formatos especiales 

La librería `pandas` nos permite importar archivos en múltiples formatos para procesarlos como un `DataFrame`. Entre estos formatos, se encuentran los archivos de tipo texto, de tipo binario y de tipo SQL. En las referencias encuentras la lista completa de métodos de lectura y escritura según el tipo de archivo. A manera de ejemplo expondremos el método `read_fwf` el cuál permite procesar archivos de ancho fijo. 

    pd.read_fwf(filepath_or_buffer, colspecs, **kwds)

* **filepath_or_buffer:** ruta del archivo de texto a procesar.  

* **colspecs:** lista de tuplas que identifica la posición donde empieza y termina cada columna. Especificando `colspecs = 'infer'` el método infiere las columnas del archivo. 

* ** **kwds:** indica que podemos especificar los parámetros disponibles en el método `read_csv`. 

##### Ejemplo 1 

Retomemos el archivo "mfh.txt" (*Major foreign holders*) el cual contiene la distribución del monto de títulos del tesoro americano de los Estados Unidos que se encuentra en cuentas afuera de su país. Dado que este es un archivo de ancho fijo, utilizaremos parte del código que aprendimos anteriormente para obtener las posiciones de inicio y de fin de las columnas que conforman el archivo.

In [1]:
with open("mfh.txt", "r") as mfh: 
    # Leemos el archivo y lo almacenamos en una sola cadena de caracteres
    mfh = mfh.read()
    # Seleccionamos la cadena de caracteres que se encuentra entre la palabra 'Period' y la palabra 'Grand'
    mfh = mfh.split('PERIOD')[1].split('Grand')[0]
    # Declaramos una lista con las líneas de nuestra cadena de caracteres
    lista_mfh = mfh.splitlines()
    # Nos quedamos únicamente con las líneas no vacías
    lista_mfh = [x for x in lista_mfh if x]
    
# Declaramos las siguientes variables: 
# Lista vacía para almacenar las posiciones de inicio de cada columna de números.
lista_inicio_cols = []
# Lista vacía para almacenar la última posición de cada columna de números. 
lista_fin_cols = []
# Cadena de caracteres que almacena la fila de guíones contenida en el archivo.
fila_guiones = lista_mfh[2]
# Variable de tipo entero para marcar el inicio de la busqueda a partir de cada columna. 
inicio_columna = 0
# Variable de tipo lógica para indicar si ya llegamos al final de la lista.
booleano = True

# Iteramos mientras el booleano se mantenga igual a `True`. 
while(booleano):
    # Declaramos el inicio de una nueva columna encontrando el siguiente guión.
    pos_inicio = inicio_columna + fila_guiones[inicio_columna:].find('-')  
    # Declaramos el ancho de la columna encontrando el siguiente espacio vacío.  
    ancho_columna = fila_guiones[pos_inicio:].find(' ')
    # Si encuentra dicho espacio
    if(ancho_columna != -1):
        # Almacenamos la posición de inicio de la columna.
        lista_inicio_cols.append(pos_inicio)
        # Declaramos la posición final de la columna.
        pos_final = pos_inicio + ancho_columna 
        # Almacenamos la posición final de la columna.
        lista_fin_cols.append(pos_final) 
        # Actualizamos el inicio de la columna para la siguiente iteración.
        inicio_columna = pos_final
    #Si no se pudo encontrar mas espacios no se se sigue iterando ya que no hay mas columnas.    
    else:
        booleano = False

FileNotFoundError: [Errno 2] No such file or directory: 'mfh.txt'

El código anterior nos permite obtener las posiciones en las que empiezan y finalizan las columnas de números. Entonces agreguemos la tupla que contiene la posición donde empieza la columna de países (posición 0) y la posición donde empieza la primera columna de números.

In [3]:
ancho_cols = [(0,lista_inicio_cols[0])]+list(zip(lista_inicio_cols, lista_fin_cols))
ancho_cols

[(0, 32),
 (32, 38),
 (40, 46),
 (48, 54),
 (56, 62),
 (64, 70),
 (72, 78),
 (80, 86),
 (88, 94),
 (96, 102),
 (104, 110),
 (112, 118),
 (120, 126)]

Importemos las librerías necesarias.

In [4]:
import numpy as np 
import pandas as pd

Luego, utilizamos el método `read_fwf` para leer el archivo, omitiendo las filas correspondientes al encabezado y al pie de tabla. Además, para efectos de este tutorial debemos omitir los registros correspondientes al gran recuento total ('Grand Total') y los demás países ('All other').

In [5]:
mfh = pd.read_fwf("mfh.txt", skiprows = range(7), skipfooter = 21, colspecs = ancho_cols, index_col = [0], header = [1,2])
# Eliminamos las dos primeras entradas ya que no contienen información relevante. 
mfh = mfh[2:-2]
mfh

,Mar,Feb,Jan,Dec,Nov,Oct,Sep,Aug,Jul,Jun,May,Apr
Country,2021,2021,2021,2020,2020,2020,2020,2020,2020,2020,2020,2020
Japan,1240.3,1258.0,1276.7,1251.3,1260.8,1273.1,1276.2,1278.4,1293.0,1261.6,1260.4,1266.5
"China, Mainland",1100.4,1104.2,1095.2,1072.3,1063.0,1054.0,1061.7,1068.0,1073.4,1074.4,1083.7,1072.8
United Kingdom,443.2,459.5,438.7,440.6,420.5,442.8,428.5,419.4,424.6,445.7,445.8,429.2
Ireland,309.8,316.0,313.6,318.1,314.2,316.3,315.7,335.2,330.7,330.2,324.2,301.3
Luxembourg,282.8,282.1,281.4,287.7,267.8,266.2,262.5,268.8,264.7,267.3,262.7,265.5
Brazil,255.5,259.0,260.4,258.3,262.2,262.9,265.1,265.0,265.7,264.1,264.4,259.5
Switzerland,254.8,257.8,254.0,255.5,253.3,255.5,255.4,253.4,250.4,247.3,243.1,241.3
Belgium,235.8,236.7,248.2,253.5,240.5,239.5,218.1,214.9,211.8,218.8,212.1,210.2
Taiwan,231.5,234.5,239.6,235.4,222.3,218.2,213.5,212.0,209.7,204.6,199.2,201.7


Al observar el `DataFrame` de la celda anterior nos damos cuenta que sería deseable rotular correctamente los niveles de las columnas y el índice, para esto podemos usar el método `rename_axis`: 

In [6]:
mfh = mfh.rename_axis("Pais")
mfh = mfh.rename_axis(["Mes","Año"], axis = 1)
mfh

Mes,Mar,Feb,Jan,Dec,Nov,Oct,Sep,Aug,Jul,Jun,May,Apr
Año,2021,2021,2021,2020,2020,2020,2020,2020,2020,2020,2020,2020
Pais,,,,,,,,,,,,
Japan,1240.3,1258.0,1276.7,1251.3,1260.8,1273.1,1276.2,1278.4,1293.0,1261.6,1260.4,1266.5
"China, Mainland",1100.4,1104.2,1095.2,1072.3,1063.0,1054.0,1061.7,1068.0,1073.4,1074.4,1083.7,1072.8
United Kingdom,443.2,459.5,438.7,440.6,420.5,442.8,428.5,419.4,424.6,445.7,445.8,429.2
Ireland,309.8,316.0,313.6,318.1,314.2,316.3,315.7,335.2,330.7,330.2,324.2,301.3
Luxembourg,282.8,282.1,281.4,287.7,267.8,266.2,262.5,268.8,264.7,267.3,262.7,265.5
Brazil,255.5,259.0,260.4,258.3,262.2,262.9,265.1,265.0,265.7,264.1,264.4,259.5
Switzerland,254.8,257.8,254.0,255.5,253.3,255.5,255.4,253.4,250.4,247.3,243.1,241.3
Belgium,235.8,236.7,248.2,253.5,240.5,239.5,218.1,214.9,211.8,218.8,212.1,210.2


## 2. Métodos para la modificación de bases de datos  

### 2.1. Método `set_index` 

Permite utilizar una o varias columnas para sustituir o modificar el índice de un `DataFrame`. Además de columnas podemos utilizar arreglos de `pandas` (`Series` e `Index`) o de arreglos de `numpy`. 

    pandas.DataFrame.set_index(keys, drop, append, inplace, verify_integrity)

* **keys:** indica la(s) llave(s) a usar como nuevo índice.

* **drop:** elimina del `DataFrame` las columnas que se emplearon como llaves en el método. Por defecto es `True`.

* **append:** preseva el índice original de la base de datos y le agrega la llave ingresada por parámetro. Por defecto es `False`.


* **inplace:** por defecto es `False`. <br><br>
    * `inplace = True`: retorna `None` y  ejecuta las operaciones sobre el dataframe oríginal. <br>
    * `inplace = False`: retorna una copia del `DataFrame` con las modificaciones. <br><br>
    
* **verify_integrity:** solo aplica si ignore_index = False. Por defecto, `verify_integrity = False`.<br><br>
    * `verify_integrity = True`: arroja ValueError si en el DataFrame resultante hay indices duplicados.<br>
    * `verify_integrity = False`: permite tener índices repetidos en el DataFrame resultante.<br><br>

**Nota:** si utilizamos un arreglo externo al dataframe, debemos asegurarnos que tenga el mismo tamaño que el que tiene actualmente el `DataFrame`.

##### Ejemplo 2

Requerimos analizar como se distribuye la tenencia de los títulos del tesoros entre dos grupos de países: los países miembros de la OECD y los países llamados paraísos fiscales. Ambos grupos no son mutuamente excluyentes por lo que pueden haber coincidencias entre ambos. A continuación, se encuentra declarada una lista para cada grupo de países.

In [7]:
lista_paraisos_fiscales = ["Belgium", "Ireland", "Luxembourg", "Malta", "Netherlands", "Cayman Island",
                           "Singapore", "Puerto Rico", "Hong Kong", "Switzerland"]

lista_oecd = ["Austria", "Australia", "Belgium", "Canada", "Chile", "Colombia", "Czech Republic", "Denmark", 
              "Estonia", "Finland", "France", "Germany", "Greece", "Hungary", "Iceland", "Ireland", "Israel", 
              "Italy", "Japan", "Korea", "Latvia", "Lithuania", "Luxembourg", "Mexico",  "Netherlands", 
              "New Zealand", "Norway", "Poland", "Portugal", "Slovak Republic", "Slovenia", "Spain", "Sweden", 
              "Switzerland", "Turkey", "United Kingdom", "United States"]

Creamos las columnas correspondientes para posteriormente añadirlas al índice.

In [8]:
# Creamos las columnas "Clasificación Fiscal" y "OECD" 
mfh.loc[:,'Clasificación_Fiscal'] = "Tributación Regular"
mfh.loc[:,'OECD'] = "No pertenece"
# Definimos cuales países son considerado paraisos fiscales y cuáles son miembros de la OECD
mfh.loc[mfh.index.isin(lista_paraisos_fiscales), 'Clasificación_Fiscal'] = "Paraiso Fiscal"
mfh.loc[mfh.index.isin(lista_oecd), 'OECD'] = "Pertenece"
# Utilizamos set_index para añadir estas columnas al índice
mfh.set_index(keys =['Clasificación_Fiscal','OECD'], append = True, inplace = True)
mfh

,,Mes,Mar,Feb,Jan,Dec,Nov,Oct,Sep,Aug,Jul,Jun,May,Apr
,,Año,2021,2021,2021,2020,2020,2020,2020,2020,2020,2020,2020,2020
Pais,Clasificación_Fiscal,OECD,,,,,,,,,,,,
Japan,Tributación Regular,Pertenece,1240.3,1258.0,1276.7,1251.3,1260.8,1273.1,1276.2,1278.4,1293.0,1261.6,1260.4,1266.5
"China, Mainland",Tributación Regular,No pertenece,1100.4,1104.2,1095.2,1072.3,1063.0,1054.0,1061.7,1068.0,1073.4,1074.4,1083.7,1072.8
United Kingdom,Tributación Regular,Pertenece,443.2,459.5,438.7,440.6,420.5,442.8,428.5,419.4,424.6,445.7,445.8,429.2
Ireland,Paraiso Fiscal,Pertenece,309.8,316.0,313.6,318.1,314.2,316.3,315.7,335.2,330.7,330.2,324.2,301.3
Luxembourg,Paraiso Fiscal,Pertenece,282.8,282.1,281.4,287.7,267.8,266.2,262.5,268.8,264.7,267.3,262.7,265.5
Brazil,Tributación Regular,No pertenece,255.5,259.0,260.4,258.3,262.2,262.9,265.1,265.0,265.7,264.1,264.4,259.5
Switzerland,Paraiso Fiscal,Pertenece,254.8,257.8,254.0,255.5,253.3,255.5,255.4,253.4,250.4,247.3,243.1,241.3
Belgium,Paraiso Fiscal,Pertenece,235.8,236.7,248.2,253.5,240.5,239.5,218.1,214.9,211.8,218.8,212.1,210.2


### 2.2. Método `reset_index`

Resetea el índice de un `DataFrame`. Si se trata de un multi-índice permite eliminar uno o más niveles, dejando la opción de preservarlos como columnas.

    pandas.DataFrame.reset_index(level, drop)

* **level:** etiqueta o índice del multi-índice a eliminar. <br><br>

* **drop:** por defecto es `False` <br><br>
`Drop = True`: se eliminan los niveles. <br><br>
`Drop = False`: los niveles pasan a ser una columna. 

### 2.3. Método `drop`

Permite eliminar filas o columnas especificando su nombre o su índice. 

    pandas.DataFrame.drop(labels, axis, index, columns, level, inplace, errors)

* **labels:** indica el nombre o la posición de las filas o columnas a eliminar.

* **axis:** por defecto es 0. <br><br>
    * `axis = 1`: indica que vamos a eliminar columnas. <br>
    * `axis = 0`: indica que vamos a eliminar filas.<br><br>


* **index:** se utiliza únicamente para eliminar filas, especificando el nombre o el índice de las filas a eliminar. Cuando utilizamos este parámetro no es necesario utilizar los parámetros `labels` y `axis`.

* **columns:** se utiliza únicamente para eliminar columnas, especificando el nombre o el índice de las columnas a eliminar. Cuando utilizamos este parámetro no es necesario utilizar los parámetros `labels` y `axis`.

* **level:** indica por nombre o por posición a cuál de los niveles del índice múltiple se le aplicará el método.


* **inplace:** por defecto es `False`. <br><br>
    * `inplace = True`: retorna `None` y  ejecuta las operaciones sobre el dataframe oríginal. <br>
    * `inplace = False`: retorna una copia del `DataFrame` con las modificaciones. <br><br>

##### Ejemplo 3

Requerimos eliminar las observaciones del `DataFrame` `mfh` correspondientes al año 2020, y almacenar el `DataFrame` resultante en un `DataFrame` llamado `mfh_2021`.

Procedemos entonces utilizando el método `drop`.

In [9]:
mfh_2021 = mfh.drop(labels = '2020', axis = 1, level = 1 )  
mfh_2021

,,Mes,Mar,Feb,Jan
,,Año,2021,2021,2021
Pais,Clasificación_Fiscal,OECD,,,
Japan,Tributación Regular,Pertenece,1240.3,1258.0,1276.7
"China, Mainland",Tributación Regular,No pertenece,1100.4,1104.2,1095.2
United Kingdom,Tributación Regular,Pertenece,443.2,459.5,438.7
Ireland,Paraiso Fiscal,Pertenece,309.8,316.0,313.6
Luxembourg,Paraiso Fiscal,Pertenece,282.8,282.1,281.4
Brazil,Tributación Regular,No pertenece,255.5,259.0,260.4
Switzerland,Paraiso Fiscal,Pertenece,254.8,257.8,254.0
Belgium,Paraiso Fiscal,Pertenece,235.8,236.7,248.2


### 2.4. Método `Groupby`

El método groupby permite realizar las siguientes tres operaciones a la vez: 
* (1) Separar en grupos el `DataFrame` de acuerdo a un criterio.
* (2) Aplicar una función a cada grupo. 
* (3) Combinar los resultados en un nuevo `DataFrame`.

La sintaxis para usar groupby es la siguiente:

    pandas.DataFrame.groupby(by, axis, level)

* **by:** indica el criterio para realizar el paso (1). Puede usarse un diccionario, un `Series`, entre otros.

* **axis:** por defecto es 0. <br><br>
    * `axis = 1`: indica que vamos a agrupar columnas. <br>
    * `axis = 0`: indica que vamos a agrupar filas.<br><br>

* **level:** indica por nombre o por posición a cuál de los niveles del índice múltiple se le aplicará el método.

##### Ejemplo 4

Requerimos un `DataFrame` llamado `mfh_resumen` que sume el total de títulos del tesoro para los niveles del índice múltiple. 

Transformamos los valores de nuestro `DataFrame` a `float` y después aplicamos el método `groupby`.

In [10]:
mfh = mfh.astype(float)
mfh_resumen = mfh.groupby(level = ['Clasificación_Fiscal','OECD']).sum()
mfh_resumen

Mes                                   Mar     Feb     Jan     Dec     Nov  \
Año                                  2021    2021    2021    2020    2020   
Clasificación_Fiscal OECD                                                   
Paraiso Fiscal       No pertenece   397.2   386.4   388.2   392.2   387.6   
                     Pertenece     1149.0  1158.7  1167.0  1184.5  1147.5   
Tributación Regular  No pertenece  2446.6  2482.2  2474.4  2449.4  2447.6   
                     Pertenece     2552.2  2583.4  2596.4  2543.0  2561.2   

Mes                                   Oct     Sep     Aug     Jul     Jun  \
Año                                  2020    2020    2020    2020    2020   
Clasificación_Fiscal OECD                                                   
Paraiso Fiscal       No pertenece   388.6   400.7   407.0   421.3   412.7   
                     Pertenece     1147.2  1120.9  1140.8  1128.8  1133.0   
Tributación Regular  No pertenece  2425.2  2432.7  2421.1  2400.4  2376.4   
                     Pertenece     2590.9  2581.7  2579.9  2597.5  2587.8   

Mes                                   May     Apr  
Año                                  2020    2020  
Clasificación_Fiscal OECD                          
Paraiso Fiscal       No pertenece   411.1   405.5  
                     Pertenece     1110.0  1084.5  
Tributación Regular  No pertenece  2351.6  2318.6  
                     Pertenece     2564.6  2562.7

### 2.5. Método `apply`

El método `apply` lleva a cabo una función bien sea sobre las columnas o sobre las filas de un `DataFrame`. 

    pandas.DataFrame.apply(func, axis)

* **func:** función a aplicar. <br><br>

* **axis:**  por defecto es 0. <br><br>

    * `axis = 1`: indica que vamos a aplicar la función sobre las columnas. <br>
    * `axis = 0`: indica que vamos a aplicar la función sobre las filas.<br><br>

A continuación explicaremos tres ventajas que presenta el método `apply`: 

#### Trabajar con funciones declaradas previamente 

El parámetro `func` puede tomar el valor de una función declarada previamente.

##### Ejemplo 5

Requerimos calcular el porcentaje de títulos distribuido en las cuentas de los países clasificados como paraíso fiscal y guardar este resultado en un `DataFrame` llamado `mfh_paraisos_fiscales`. 

Para esto declaramos la función `calcular_porcentaje`.

In [11]:
def calcular_porcentaje(lista):
    
    rta = lista / sum(lista)
    
    return rta 

Después la usamos como parámetro del método `apply`.

In [12]:
# Declaramos el Segmentador de Índices
idx = pd.IndexSlice
mfh_paraisos_fiscales = mfh.loc[idx[:,'Paraiso Fiscal',],]
mfh_paraisos_fiscales = mfh_paraisos_fiscales.apply(calcular_porcentaje)
mfh_paraisos_fiscales

,,Mes,Mar,Feb,Jan,Dec,Nov,Oct,Sep,Aug,Jul,Jun,May,Apr
,,Año,2021,2021,2021,2020,2020,2020,2020,2020,2020,2020,2020,2020
Pais,Clasificación_Fiscal,OECD,,,,,,,,,,,,
Ireland,Paraiso Fiscal,Pertenece,0.200362,0.204518,0.201646,0.201750,0.204677,0.205951,0.207479,0.216565,0.213341,0.213625,0.213135,0.202215
Luxembourg,Paraiso Fiscal,Pertenece,0.182900,0.182577,0.180941,0.182470,0.174451,0.173330,0.172516,0.173666,0.170763,0.172931,0.172704,0.178188
Switzerland,Paraiso Fiscal,Pertenece,0.164791,0.166850,0.163323,0.162047,0.165006,0.166363,0.167850,0.163716,0.161538,0.159992,0.159819,0.161946
Belgium,Paraiso Fiscal,Pertenece,0.152503,0.153194,0.159594,0.160779,0.156667,0.155945,0.143336,0.138842,0.136636,0.141554,0.139439,0.141074
Hong Kong,Paraiso Fiscal,No pertenece,0.146812,0.145622,0.143969,0.142132,0.145789,0.147545,0.158846,0.159388,0.169473,0.169632,0.176846,0.174094
Singapore,Paraiso Fiscal,No pertenece,0.110076,0.104459,0.105646,0.106615,0.106703,0.105482,0.104495,0.103566,0.102316,0.097367,0.093419,0.098054
Netherlands,Paraiso Fiscal,Pertenece,0.042556,0.042780,0.044882,0.044206,0.046707,0.045384,0.045478,0.044256,0.045933,0.044899,0.044639,0.044430


#### Encadenar métodos 

En `pandas` podemos encadenar métodos, esto significa que se puede invocar un método tras otro sin redefinir el `DataFrame` repetidamente. El método `apply` puede encadenarse con otros métodos, lo que resulta práctico en términos de ahorro de líneas de código y claridad de lectura. 

##### Ejemplo 6

Filtra la base para solo trabajar con aquellos países que son parte de la OECD, de tal forma que pueda calcular el porcentaje de títulos bajo posesión de cada país.

Hacemos entonces el filtrado y directamente realizamos el `apply`.

In [13]:
mfh_oecd = mfh.loc[idx[:,:,'Pertenece'],].apply(calcular_porcentaje,axis = 0)
mfh_oecd

,,Mes,Mar,Feb,Jan,Dec,Nov,Oct,Sep,Aug,Jul,Jun,May,Apr
,,Año,2021,2021,2021,2020,2020,2020,2020,2020,2020,2020,2020,2020
Pais,Clasificación_Fiscal,OECD,,,,,,,,,,,,
Japan,Tributación Regular,Pertenece,0.335108,0.336175,0.339241,0.335694,0.339957,0.340574,0.344677,0.343591,0.346993,0.339067,0.343003,0.347253
United Kingdom,Tributación Regular,Pertenece,0.119745,0.122792,0.116570,0.118203,0.113382,0.118456,0.115729,0.112721,0.113947,0.119786,0.121319,0.117679
Ireland,Paraiso Fiscal,Pertenece,0.083703,0.084445,0.083329,0.085339,0.084720,0.084615,0.085264,0.090091,0.088748,0.088744,0.088227,0.082611
Luxembourg,Paraiso Fiscal,Pertenece,0.076408,0.075385,0.074773,0.077183,0.072209,0.071213,0.070896,0.072244,0.071036,0.071839,0.071491,0.072796
Switzerland,Paraiso Fiscal,Pertenece,0.068843,0.068892,0.067492,0.068545,0.068299,0.068350,0.068979,0.068105,0.067198,0.066464,0.066157,0.066160
Belgium,Paraiso Fiscal,Pertenece,0.063709,0.063253,0.065951,0.068008,0.064848,0.064070,0.058905,0.057758,0.056839,0.058805,0.057721,0.057633
Korea,Tributación Regular,Pertenece,0.033205,0.033591,0.033268,0.035117,0.033111,0.031861,0.033274,0.032978,0.033035,0.032977,0.031922,0.032902
France,Tributación Regular,Pertenece,0.031044,0.029048,0.032417,0.029859,0.035700,0.036623,0.036488,0.035827,0.034887,0.038728,0.035514,0.037755


#### Múltiples operaciones simultaneamente

El método `apply` permite ejecutar varias operaciones a la vez, lo cual nos ahorra líneas de código o encadenar más métodos de los necesarios. 

##### Ejemplo 7

Requerimos nuevamente filtrar la base para solo trabajar con aquellos países que son parte de la OECD, de tal forma que calculemos el porcentaje de títulos bajo posesión de cada país. Esta vez, redondearemos el resultado a tres cifras decimales. 

Una manera de solucionar este requerimiento es utilizar la división de objetos y el método `round` de `pandas`.

In [14]:
mfh_oecd = mfh.loc[idx[:,:,'Pertenece'],]
mfh_oecd_norm = (mfh_oecd / mfh_oecd.sum()).round(3)
mfh_oecd_norm

,,Mes,Mar,Feb,Jan,Dec,Nov,Oct,Sep,Aug,Jul,Jun,May,Apr
,,Año,2021,2021,2021,2020,2020,2020,2020,2020,2020,2020,2020,2020
Pais,Clasificación_Fiscal,OECD,,,,,,,,,,,,
Japan,Tributación Regular,Pertenece,0.335,0.336,0.339,0.336,0.340,0.341,0.345,0.344,0.347,0.339,0.343,0.347
United Kingdom,Tributación Regular,Pertenece,0.120,0.123,0.117,0.118,0.113,0.118,0.116,0.113,0.114,0.120,0.121,0.118
Ireland,Paraiso Fiscal,Pertenece,0.084,0.084,0.083,0.085,0.085,0.085,0.085,0.090,0.089,0.089,0.088,0.083
Luxembourg,Paraiso Fiscal,Pertenece,0.076,0.075,0.075,0.077,0.072,0.071,0.071,0.072,0.071,0.072,0.071,0.073
Switzerland,Paraiso Fiscal,Pertenece,0.069,0.069,0.067,0.069,0.068,0.068,0.069,0.068,0.067,0.066,0.066,0.066
Belgium,Paraiso Fiscal,Pertenece,0.064,0.063,0.066,0.068,0.065,0.064,0.059,0.058,0.057,0.059,0.058,0.058
Korea,Tributación Regular,Pertenece,0.033,0.034,0.033,0.035,0.033,0.032,0.033,0.033,0.033,0.033,0.032,0.033
France,Tributación Regular,Pertenece,0.031,0.029,0.032,0.030,0.036,0.037,0.036,0.036,0.035,0.039,0.036,0.038


En cambio podríamos haber usado una sola línea de código para cubrir este requerimiento.

In [15]:
mfh_oecd_norm = mfh.loc[idx[:,:,'Pertenece'],].apply(lambda x: round(x / x.sum(),3))
mfh_oecd_norm

,,Mes,Mar,Feb,Jan,Dec,Nov,Oct,Sep,Aug,Jul,Jun,May,Apr
,,Año,2021,2021,2021,2020,2020,2020,2020,2020,2020,2020,2020,2020
Pais,Clasificación_Fiscal,OECD,,,,,,,,,,,,
Japan,Tributación Regular,Pertenece,0.335,0.336,0.339,0.336,0.340,0.341,0.345,0.344,0.347,0.339,0.343,0.347
United Kingdom,Tributación Regular,Pertenece,0.120,0.123,0.117,0.118,0.113,0.118,0.116,0.113,0.114,0.120,0.121,0.118
Ireland,Paraiso Fiscal,Pertenece,0.084,0.084,0.083,0.085,0.085,0.085,0.085,0.090,0.089,0.089,0.088,0.083
Luxembourg,Paraiso Fiscal,Pertenece,0.076,0.075,0.075,0.077,0.072,0.071,0.071,0.072,0.071,0.072,0.071,0.073
Switzerland,Paraiso Fiscal,Pertenece,0.069,0.069,0.067,0.069,0.068,0.068,0.069,0.068,0.067,0.066,0.066,0.066
Belgium,Paraiso Fiscal,Pertenece,0.064,0.063,0.066,0.068,0.065,0.064,0.059,0.058,0.057,0.059,0.058,0.058
Korea,Tributación Regular,Pertenece,0.033,0.034,0.033,0.035,0.033,0.032,0.033,0.033,0.033,0.033,0.032,0.033
France,Tributación Regular,Pertenece,0.031,0.029,0.032,0.030,0.036,0.037,0.036,0.036,0.035,0.039,0.036,0.038


### 2.7. Método `rename`

El método `rename` sirve para renombrar columnas o índices. 

    pandas.DataFrame.rename(mapper, index, columns, axis, inplace, level)

* **mapper:** diccionario o función para renombrar las columnas o índices. <br><br>

* **index:** diccionario o función para renombrar índices. Cuando utilizamos este parámetro no es necesario utilizar los parámetros `mapper` y `axis`. <br><br>

* **columns:** diccionario o función para renombrar columnas. Cuando utilizamos este parámetro no es necesario utilizar los parámetros `mapper` y `axis`. <br><br>

* **axis:** por defecto es 0. <br><br>
    * `axis = 1`:  indica que vamos a renombrar columnas.<br>
    * `axis = 0`: indica que vamos a renombrar filas.<br>

* **inplace:** por defecto es `False`. <br><br>
    * `inplace = True`: retorna `None` y  ejecuta las operaciones sobre el dataframe oríginal. <br>
    * `inplace = False`: retorna una copia del `DataFrame` con las modificaciones. <br><br>

* **level:** indica por nombre o por posición a cuál de los niveles del índice múltiple se le debe aplicar el diccionario o la función. 

##### Ejemplo 8

En la siguiente celda encuentras un diccionario con los nombres de los países en inglés y en español.

In [ ]:
df_paises = pd.read_csv("Archivos\DiccionarioPaises.csv", encoding = 'utf-8-sig', sep = ";", index_col = [1])
dicc_paises = df_paises.to_dict()['País en español']

A continuación encuentras un diccionario con las abreviaturas de los meses en inglés y en español.

In [21]:
dicc_meses = {'Jan':'ENE', 'Feb': 'FEB', 'Mar':'MAR', 'Apr':'ABR', 'May':'MAY' ,'Jun': 'JUN', 'Jul':'JUL',
                    'Aug': 'AGO', 'Sep': 'SEP', 'Oct':'OCT','Nov':'NOV', 'Dec':'DIC'}

Requerimos traducir al español los meses y países del `DataFrame` utilizando los diccionarios provistos 

In [22]:
mfh = mfh.rename(mapper = dicc_paises)
mfh = mfh.rename(columns = dicc_meses) 
mfh

,,Mes,MAR,FEB,ENE,DIC,NOV,OCT,SEP,AGO,JUL,JUN,MAY,ABR
,,Año,2021,2021,2021,2020,2020,2020,2020,2020,2020,2020,2020,2020
Pais,Clasificación_Fiscal,OECD,,,,,,,,,,,,
Japón,Tributación Regular,Pertenece,1240.3,1258.0,1276.7,1251.3,1260.8,1273.1,1276.2,1278.4,1293.0,1261.6,1260.4,1266.5
China,Tributación Regular,No pertenece,1100.4,1104.2,1095.2,1072.3,1063.0,1054.0,1061.7,1068.0,1073.4,1074.4,1083.7,1072.8
Reino Unido,Tributación Regular,Pertenece,443.2,459.5,438.7,440.6,420.5,442.8,428.5,419.4,424.6,445.7,445.8,429.2
Irlanda,Paraiso Fiscal,Pertenece,309.8,316.0,313.6,318.1,314.2,316.3,315.7,335.2,330.7,330.2,324.2,301.3
Luxemburgo,Paraiso Fiscal,Pertenece,282.8,282.1,281.4,287.7,267.8,266.2,262.5,268.8,264.7,267.3,262.7,265.5
Brasil,Tributación Regular,No pertenece,255.5,259.0,260.4,258.3,262.2,262.9,265.1,265.0,265.7,264.1,264.4,259.5
Suiza,Paraiso Fiscal,Pertenece,254.8,257.8,254.0,255.5,253.3,255.5,255.4,253.4,250.4,247.3,243.1,241.3
Bélgica,Paraiso Fiscal,Pertenece,235.8,236.7,248.2,253.5,240.5,239.5,218.1,214.9,211.8,218.8,212.1,210.2


### 2.8. Método `replace`

El método `replace` usa la siguiente sintaxis para reemplazar un valor por otro.

    pandas.DataFrame.replace( to_replace, value, inplace) 

* **to_replace:** valor específico, estructura de datos. <br><br>

* **value:** valor que sustituirá el valor a reemplazar.<br><br>

* **inplace:** por defecto es `False`. <br><br>
    * `inplace = True`: retorna `None` y  ejecuta las operaciones sobre el dataframe oríginal. <br>
    * `inplace = False`: retorna una copia del `DataFrame` con las modificaciones. <br><br>

### 2.9. Método `stack`

El método `stack` transfiere uno o varios niveles de las columnas al índice. Al emplearlo, el número de niveles del índice se va a ver aumentando y el número de niveles de las columnas se va a ver disminuido. 

    pandas.DataFrame.stack(level, fill_value) 

* **level:** nombre o índice del nivel a transferir. Por defecto es -1 (el último nivel) 

* **fill_value:** valor para reemplazar los datos faltantes que se generen. 

##### Ejemplo 9

Pasa el nivel correspondiente a los meses al índice de `mfh_2021`.

Procedemos entonces usando el método `stack`.

In [19]:
mfh_2021 = mfh_2021.stack("Mes") 
mfh_2021

Año                                                           2021
Pais                 Clasificación_Fiscal OECD         Mes        
Japan                Tributación Regular  Pertenece    Feb  1258.0
                                                       Jan  1276.7
                                                       Mar  1240.3
China, Mainland      Tributación Regular  No pertenece Feb  1104.2
                                                       Jan  1095.2
                                                       Mar  1100.4
United Kingdom       Tributación Regular  Pertenece    Feb   459.5
                                                       Jan   438.7
                                                       Mar   443.2
Ireland              Paraiso Fiscal       Pertenece    Feb   316.0
                                                       Jan   313.6
                                                       Mar   309.8
Luxembourg           Paraiso Fiscal       Pertenece    Feb   282.1
                                                       Jan   281.4
                                                       Mar   282.8
Brazil               Tributación Regular  No pertenece Feb   259.0
                                                       Jan   260.4
                                                       Mar   255.5
Switzerland          Paraiso Fiscal       Pertenece    Feb   257.8
                                                       Jan   254.0
                                                       Mar   254.8
Belgium              Paraiso Fiscal       Pertenece    Feb   236.7
                                                       Jan   248.2
                                                       Mar   235.8
Taiwan               Tributación Regular  No pertenece Feb   234.5
                                                       Jan   239.6
                                                       Mar   231.5
Hong Kong            Paraiso Fiscal       No pertenece Feb   225.0
                                                       Jan   223.9
                                                       Mar   227.0
...                                                            ...
Philippines          Tributación Regular  No pertenece Feb    50.8
                                                       Jan    48.0
                                                       Mar    50.9
Mexico               Tributación Regular  Pertenece    Feb    49.4
                                                       Jan    49.8
                                                       Mar    48.0
Kuwait               Tributación Regular  No pertenece Feb    44.8
                                                       Jan    45.3
                                                       Mar    45.7
United Arab Emirates Tributación Regular  No pertenece Feb    50.6
                                                       Jan    33.8
                                                       Mar    45.6
Australia            Tributación Regular  Pertenece    Feb    40.5
                                                       Jan    41.0
                                                       Mar    41.3
Sweden               Tributación Regular  Pertenece    Feb    40.8
                                                       Jan    42.2
                                                       Mar    40.0
Chile                Tributación Regular  Pertenece    Feb    38.2
                                                       Jan    30.1
                                                       Mar    38.3
Vietnam              Tributación Regular  No pertenece Feb    37.3
                                                       Jan    37.4
                                                       Mar    37.8
Italy                Tributación Regular  Pertenece    Feb    36.8
                                                       Jan    38.3
                                               

### 2.10. Método `unstack`

El método `unstack` transfiere uno o varios niveles del índice a las columna. Al emplearlo, el número de niveles de las columnas se va a ver aumentando y el número de niveles del índice se va a ver disminuido. 

    pandas.DataFrame.unstack(level, fill_value) 

* **level:** nombre o índice del nivel a transferir. Por defecto es -1 (el último nivel) 

* **fill_value:** valor para reemplazar los datos faltantes que se generen. 

#### Ejemplo 10

Se te solicita revertir el cambio realizado en el ejemplo anterior.

Utilizamos el método `unstack` para revertir el cambio realizado.

In [20]:
mfh_2021 = mfh_2021.unstack("Mes")
mfh_2021

Año                                                       2021                
Mes                                                        Feb     Jan     Mar
Pais                 Clasificación_Fiscal OECD                                
Australia            Tributación Regular  Pertenece       40.5    41.0    41.3
Belgium              Paraiso Fiscal       Pertenece      236.7   248.2   235.8
Bermuda              Tributación Regular  No pertenece    67.1    68.2    66.3
Brazil               Tributación Regular  No pertenece   259.0   260.4   255.5
Canada               Tributación Regular  Pertenece      112.7   126.7   106.0
Cayman Islands       Tributación Regular  No pertenece   216.0   217.6   215.2
Chile                Tributación Regular  Pertenece       38.2    30.1    38.3
China, Mainland      Tributación Regular  No pertenece  1104.2  1095.2  1100.4
Colombia             Tributación Regular  Pertenece       30.6    32.4    35.2
France               Tributación Regular  Pertenece      108.7   122.0   114.9
Germany              Tributación Regular  Pertenece       73.5    74.8    77.6
Hong Kong            Paraiso Fiscal       No pertenece   225.0   223.9   227.0
India                Tributación Regular  No pertenece   204.4   211.6   200.0
Ireland              Paraiso Fiscal       Pertenece      316.0   313.6   309.8
Israel               Tributación Regular  Pertenece       58.2    53.7    61.0
Italy                Tributación Regular  Pertenece       36.8    38.3    36.5
Japan                Tributación Regular  Pertenece     1258.0  1276.7  1240.3
Korea                Tributación Regular  Pertenece      125.7   125.2   122.9
Kuwait               Tributación Regular  No pertenece    44.8    45.3    45.7
Luxembourg           Paraiso Fiscal       Pertenece      282.1   281.4   282.8
Mexico               Tributación Regular  Pertenece       49.4    49.8    48.0
Netherlands          Paraiso Fiscal       Pertenece       66.1    69.8    65.8
Norway               Tributación Regular  Pertenece       97.6    91.5    94.4
Philippines          Tributación Regular  No pertenece    50.8    48.0    50.9
Poland               Tributación Regular  Pertenece       53.2    53.3    52.6
Saudi Arabia         Tributación Regular  No pertenece   132.9   135.1   130.8
Singapore            Paraiso Fiscal       No pertenece   161.4   164.3   170.2
Sweden               Tributación Regular  Pertenece       40.8    42.2    40.0
Switzerland          Paraiso Fiscal       Pertenece      257.8   254.0   254.8
Taiwan               Tributación Regular  No pertenece   234.5   239.6   231.5
Thailand             Tributación Regular  No pertenece    80.6    82.2    66.9
United Arab Emirates Tributación Regular  No pertenece    50.6    33.8    45.6
United Kingdom       Tributación Regular  Pertenece      459.5   438.7   443.2
Vietnam              Tributación Regular  No pertenece    37.3    37.4    37.8

## Referencias

Pandas (2021). Documentación sobre I/O tools (text, CSV, HDF5...). Recuperado el 24 de Abril de 2020 de: https://pandas.pydata.org/pandas-docs/stable/user_guide/io.html 

Pandas (2020). Documentación sobre los objetos índices. Recuperado el 15 de mayo de 2020 de: https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.IndexSlice.html

Pandas (2020). Documentación sobre los métodos DataFrame. Recuperado el 15 de mayo de 2020 de: https://pandas.pydata.org/docs/reference/frame.html

Departamento del Tesoro Americano (2020). Principales tenedores foráneos de títulos del tesoro [Base de Datos]. Recuperado el  15 de septiembre de 2020 de : 
https://ticdata.treasury.gov/Publish/mfh.txt.

## Créditos

**Autor(es):** Jorge Esteban Camargo Forero, Diego Alejandro Cely Gómez

**Fecha Última actualización:** 06/08/2021